In [4]:
#!{sys.executable} -m pip install matplotlib

In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import ipywidgets as widgets

In [115]:
def filter_df(df_orig, col, arg, op='=='):
    df = df_orig.copy()
    
    if op == '==':
        df = df.loc[df[col]==arg]
    elif op == '<':
        df = df.loc[df[col]<arg]
    elif op == '>':
        df = df.loc[df[col]>arg]
    elif op == '<=':
        df = df.loc[df[col]<=arg]
    elif op == '>=':
        df = df.loc[df[col]>=arg]
    elif op == '!=':
        df = df.loc[df[col]!=arg]
    elif op == 'isin':
        assert(type(arg) is list)
        df = df.loc[df[col].isin(arg)]
    else:
        print('op type not supported!')
        assert(1==0)
    
    return df

def get_path(path_string):
    path_list = [int(node_id) for node_id in path_string.split(sep=';')[2:]]
    return path_list

In [116]:
config_df = pd.read_csv("../cmake-build-debug/config_log.txt", sep=",", header=0)
display(config_df)
config_dict = config_df.to_dict(orient='list')


sim_df = pd.read_csv("../cmake-build-debug/sim_log.txt", sep=",", header=0)
display(sim_df.head())

sampling_df = pd.read_csv("../cmake-build-debug/sampling_log.txt", sep=",", header=0)
display(sampling_df)

,sim_time,rollouts,horizon,obstacle_cost,obstacle_rad_0,obstacle_rad_1,obstacle_rad_2,obstacle_rad_3,obstacle_pos_0_x,obstacle_pos_0_y,obstacle_pos_1_x,obstacle_pos_1_y,obstacle_pos_2_x,obstacle_pos_2_y,obstacle_pos_3_x,obstacle_pos_3_y,target_state_0,target_state_1,Unnamed: 18
0,150,20,20,10000,10,5,10,5,5,20,55,15,40,20,80,40,100,50,NaN


,simstep,state_0,state_1,state_2,state_3,state_4,state_5,Unnamed: 7
0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
1,1,0.421516,0.096283,0.843033,0.192565,0.843033,0.192565,NaN
2,2,1.299690,0.445144,0.913307,0.505159,0.070275,0.312594,NaN
3,3,2.311800,0.823767,1.110930,0.252087,0.197622,-0.253071,NaN
4,4,3.384930,1.143170,1.035320,0.386708,-0.075605,0.134621,NaN


,node_id,simstep,step,rollout,state_0,state_1,state_2,state_3,state_4,state_5,cost_cum,path_to_leaf,Unnamed: 12
0,22,0,0,0,0.067206,-0.076940,0.134412,-0.153881,0.134412,-0.153881,223.5810,0;1;22,NaN
1,23,0,0,1,-0.042592,0.058205,-0.085183,0.116411,-0.085183,0.116411,223.6190,0;2;23,NaN
2,24,0,0,2,-0.018423,-0.117294,-0.036847,-0.234588,-0.036847,-0.234588,223.6760,0;3;24,NaN
3,25,0,0,3,0.035822,-0.042862,0.071643,-0.085723,0.071643,-0.085723,223.5940,0;4;25,NaN
4,26,0,0,4,-0.001366,-0.008443,-0.002733,-0.016885,-0.002733,-0.016885,223.6120,0;5;26,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,60017,149,19,15,98.262600,52.069500,0.080938,0.124313,-0.122441,-0.165209,26.2292,0;6;59627;59647;59668;59695;59715;59730;59753;...,NaN
59996,60018,149,19,16,97.847000,50.907100,-0.109158,-0.075316,-0.137406,0.095430,25.4642,0;6;59627;59647;59668;59695;59715;59730;59753;...,NaN
59997,60019,149,19,17,98.043600,51.112600,-0.201089,0.299048,-0.408613,0.306111,24.9717,0;6;59627;59647;59668;59695;59715;59730;59753;...,NaN
59998,60020,149,19,18,98.351800,50.832900,0.415318,-0.260344,0.207794,-0.253281,24.5678,0;6;59627;59647;59668;59695;59715;59730;59753;...,NaN


In [117]:
#df_timestep = filter_df(sampling_df, "simstep", 10)
#df_timestep = filter_df(df_timestep, "step", 3)

In [118]:
def draw_interactive(simstep, step):
      
    fig, ax = fig, ax = plt.subplots(1, figsize=(15,10))
   
    # Add obstacles
    if config_dict['obstacle_cost'][0] > 0:
        for obstacle_nr in range(0,4):
            obst_id_rad = 'obstacle_rad_'+str(obstacle_nr)
            obst_id_x = 'obstacle_pos_'+str(obstacle_nr)+'_x'
            obst_id_y = 'obstacle_pos_'+str(obstacle_nr)+'_y'
            
            ellipse_plot_0 = patches.Circle((config_dict[obst_id_x][0],config_dict[obst_id_y][0]), 
                                     radius=config_dict[obst_id_rad][0], color='gray')
      
            ax.add_patch(ellipse_plot_0)
    
    # filter data down to relevant data
    df_timestep = filter_df(sampling_df, "simstep", simstep, op='==')
    df_timestep = filter_df(df_timestep, "step", step, op='==')

    # plot path of different rollouts
    for idx, row in df_timestep.iterrows():
        path = get_path(row['path_to_leaf'])
        df_path = filter_df(sampling_df, "node_id", path, op='isin')
        
        ax.plot(df_path[['state_0']], df_path[['state_1']], marker='.', zorder=10)
    
    
    ax.plot(sim_df[['state_0']][:simstep+1], sim_df[['state_1']][:simstep+1], c='r')
    ax.plot(sim_df[['state_0']], sim_df[['state_1']], c='r', alpha=0.3)
    ax.scatter(sim_df[['state_0']].iloc[simstep], sim_df[['state_1']].iloc[simstep],s=60, c='r',zorder=20)


    padding = 10;
    ax.set_xlim(sim_df[['state_0']].min()[0]-padding, sim_df[['state_0']].max()[0]+padding)
    ax.set_ylim(sim_df[['state_1']].min()[0]-padding, sim_df[['state_1']].max()[0]+padding)
    plt.show()
    
widgets.interactive(draw_interactive, 
                    simstep = widgets.IntSlider(value=0, min=0, max=config_dict['sim_time'][0]-1, step=1), 
                    step= widgets.IntSlider(value=19, min=0, max=config_dict['horizon'][0]-1))

interactive(children=(IntSlider(value=0, description='simstep', max=149), IntSlider(value=19, description='ste…

In [109]:
get_path(sampling_df.loc[23, 'path_to_leaf'])


simstep = 1
step = 1
df_timestep = filter_df(sampling_df, "simstep", simstep, op='==')
df_timestep = filter_df(df_timestep, "step", step, op='==')

for idx, row in df_timestep.iterrows():
    path = get_path(row['path_to_leaf'])
    df_path = filter_df(sampling_df, "node_id", path, op='isin')
    

In [91]:
sim_df[['state_0']].min()[0]

0.0